# Машинное обучение, ФКН ВШЭ

## Практическое задание 3. Градиентный спуск своими руками

### Общая информация
Дата выдачи: 05.10.2019

Мягкий дедлайн: 07:59MSK 15.10.2019 (за каждый день просрочки снимается 1 балл)

Жесткий дедлайн: 23:59MSK 17.10.2019

### О задании

В данном задании необходимо реализовать обучение линейной регрессии с помощью различных вариантов градиентного спуска.


### Оценивание и штрафы
Каждая из задач имеет определенную «стоимость» (указана в скобках около задачи). Максимально допустимая оценка за работу — 10 баллов.

Сдавать задание после указанного срока сдачи нельзя. При выставлении неполного балла за задание в связи с наличием ошибок на усмотрение проверяющего предусмотрена возможность исправить работу на указанных в ответном письме условиях.

Задание выполняется самостоятельно. «Похожие» решения считаются плагиатом и все задействованные студенты (в том числе те, у кого списали) не могут получить за него больше 0 баллов (подробнее о плагиате см. на странице курса). Если вы нашли решение какого-то из заданий (или его часть) в открытом источнике, необходимо указать ссылку на этот источник в отдельном блоке в конце вашей работы (скорее всего вы будете не единственным, кто это нашел, поэтому чтобы исключить подозрение в плагиате, необходима ссылка на источник).

Неэффективная реализация кода может негативно отразиться на оценке.

Все ответы должны сопровождаться кодом или комментариями о том, как они были получены.


### Формат сдачи
Задания сдаются через систему Anytask. Инвайт можно найти на странице курса. Присылать необходимо ноутбук с выполненным заданием. 

Для удобства проверки самостоятельно посчитайте свою максимальную оценку (исходя из набора решенных задач) и укажите ниже.

**Оценка**: ...

## Реализация градиентного спуска

Реализуйте линейную регрессию с функцией потерь MSE, обучаемую с помощью:

** Задание 1 (1 балл)** Градиентного спуска;

** Задание 2 (1.5 балла)** Стохастического градиентного спуска;

** Задание 3 (2.5 балла)** Метода Momentum.


Во всех пунктах необходимо соблюдать следующие условия:

* Все вычисления должны быть векторизованы;
* Циклы средствами python допускается использовать только для итераций градиентного спуска;
* В качестве критерия останова необходимо использовать (одновременно):

    * проверку на евклидовую норму разности весов на двух соседних итерациях (например, меньше некоторого малого числа порядка $10^{-6}$, задаваемого параметром `tolerance`);
    * достижение максимального числа итераций (например, 10000, задаваемого параметром `max_iter`).
* Чтобы проследить, что оптимизационный процесс действительно сходится, будем использовать атрибут класса `loss_history` — в нём после вызова метода `fit` должны содержаться значения функции потерь для всех итераций, начиная с первой (до совершения первого шага по антиградиенту);
* Инициализировать веса можно случайным образом или нулевым вектором. 


Ниже приведён шаблон класса, который должен содержать код реализации каждого из методов.

In [110]:
import numpy as np
from sklearn.base import BaseEstimator
from sklearn.metrics import mean_squared_error

class LinearReg(BaseEstimator):
    def __init__(self, gd_type='stochastic', 
                 tolerance=1e-4, max_iter=1000, w0=None, alpha=1e-3, eta=1e-2, batch_size=1):
        """
        gd_type: 'full' or 'stochastic' or 'momentum'
        tolerance: for stopping gradient descent
        max_iter: maximum number of steps in gradient descent
        w0: np.array of shape (d) - init weights
        w: weights after training
        eta: learning rate
        alpha: momentum coefficient
        """
        self.gd_type = gd_type
        self.tolerance = tolerance
        self.max_iter = max_iter
        self.w0 = w0
        self.alpha = alpha
        self.w = None
        self.eta = eta
        self.batch_size = batch_size
        self.loss_history = None # list of loss function values at each training iteration
    
    def fit(self, X, y):
        """
        X: np.array of shape (ell, d)
        y: np.array of shape (ell)
        ---
        output: self
        """
        ell, d = X.shape
        #print(d)
        if not self.w0:
            old_w = np.zeros(d,)
            self.w = np.zeros(d,)
        else:
            old_w = self.w0
            self.w = self.w0
        self.loss_history = []
        #╰( ͡° ͜ʖ ͡° )つ──☆*:・ﾟ
        for i in range(self.max_iter):
            old_w, self.w = self.w, self.iterate(X, y)
            self.loss_history.append(self.calc_loss(X, y))
            if i % 100 == 0:
                print(i, self.calc_loss(X, y))
            if np.linalg.norm(self.w - old_w) < self.tolerance:
                return self
        return self
    
    def iterate(self, X, y):
        raise NotImplementedError()
    
    def predict(self, X):
        if self.w is None:
            raise Exception('Not trained yet')
        return X.dot(self.w)
    
    def calc_gradient(self, X, y):
        """
        X: np.array of shape (ell, d) (ell can be equal to 1 if stochastic)
        y: np.array of shape (ell)
        ---
        output: np.array of shape (d)
        """
        #╰( ͡° ͜ʖ ͡° )つ──☆*:・ﾟ
        return 2 * X.T.dot(X.dot(self.w) - y)

    def calc_loss(self, X, y):
        """
        X: np.array of shape (ell, d)
        y: np.array of shape (ell)
        ---
        output: float 
        """ 
        #╰( ͡° ͜ʖ ͡° )つ──☆*:・ﾟ
        return mean_squared_error(self.predict(X), y)

In [111]:
class LinearRegGradientDescent(LinearReg):
    def iterate(self, X, y):
        return self.w - (self.eta * self.calc_gradient(X, y)) / y.shape[0]

In [112]:
class LinearRegStochasticGradientDescent(LinearReg):
    def iterate(self, X, y):
        sample = np.random.randint(X.shape[0], size=self.batch_size)
        #print(sample)
        #print('X', X.shape)
        #print('y', y.shape)
        return self.w - (self.eta * self.calc_gradient(X[sample], y[sample])) / y.shape[0]

In [21]:
class LinearRegMomentum(LinearReg):
    def iterate():
        raise NotImplementedError()

** Задание 4 (0 баллов)**. 
* Загрузите данные из домашнего задания 2 ([train.csv](https://www.kaggle.com/c/nyc-taxi-trip-duration/data));
* Разбейте выборку на обучающую и тестовую в отношении 7:3 с random_seed=0;
* Преобразуйте целевую переменную `trip_duration` как $\hat{y} = \log{(y + 1)}$.

In [22]:
from sklearn.model_selection import train_test_split
import pandas as pd
from sklearn.compose import ColumnTransformer, make_column_transformer
from sklearn.preprocessing import OneHotEncoder
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler
from haversine import haversine

In [23]:
def get_dataframe_for_gradient():
    dataframe = pd.read_csv('train.csv')
    dataframe.drop('dropoff_datetime', axis=1, inplace=True)
    dataframe['pickup_datetime'] = pd.to_datetime(dataframe['pickup_datetime'])
    dataframe['log_trip_duration'] = np.log1p(dataframe['trip_duration'])
    dataframe['weekday'] = dataframe['pickup_datetime'].dt.weekday
    dataframe['day'] = dataframe['pickup_datetime'].dt.day
    dataframe['month'] = dataframe['pickup_datetime'].dt.month
    dataframe['hour'] = dataframe['pickup_datetime'].dt.hour
    dataframe['dayofyear'] = dataframe['pickup_datetime'].dt.dayofyear
    dataframe['is_anomalous_1'] = (dataframe.month == 1) & ((dataframe.day == 23) | (dataframe.day == 24))
    dataframe['is_anomalous_2'] = (dataframe.month == 5) & (dataframe.day == 30)
    dataframe['is_anomalous_3'] = (dataframe.month == 1) & ((dataframe.day == 25) | (dataframe.day == 26))
    dataframe['is_anomalous_4'] = (dataframe.month == 1) & (dataframe.day == 27)
    dataframe[['is_anomalous_%d' % i for i in range(1, 5)]] = dataframe[[
        'is_anomalous_%d' % i for i in range(1, 5)]].astype(int)

    dataframe['empty_roads'] = (
          (dataframe.hour == 5)
        | ((dataframe.hour == 4) & (dataframe.weekday <= 3))
        | ((dataframe.hour == 6) & (dataframe.weekday >= 5))
        | ((dataframe.hour == 3) & ((dataframe.weekday == 1) | (dataframe.weekday == 3)))
    )

    dataframe['busy_roads'] = (
          (((dataframe.hour <= 15) & (dataframe.hour >= 9)) & ((dataframe.weekday >= 1) | (dataframe.weekday <= 3)))
        | (((dataframe.hour <= 11) & (dataframe.hour >= 9)) & (dataframe.weekday == 4))
    )

    row_modifier = lambda p_lat, p_lon, d_lat, d_lon: haversine((p_lat, p_lon), (d_lat, d_lon))

    dataframe['haversine'] = np.vectorize(row_modifier)(dataframe.pickup_latitude, dataframe.pickup_longitude, 
        dataframe.dropoff_latitude, dataframe.dropoff_longitude)
    dataframe['log_haversine'] = np.log1p(dataframe['haversine'])

    row_modifier = lambda p_lat, p_lon, d_lat, d_lon: haversine((p_lat, p_lon), (d_lat, d_lon))

    LA_GUARDIA_LAT = 40.7769
    LA_GUARDIA_LON = -73.8740
    J_KENNEDY_LAT = 40.6413
    J_KENNEDY_LON = -73.7781
    dataframe['pickup_distance_from_la_guardia'] = np.vectorize(row_modifier)(dataframe.pickup_latitude, dataframe.pickup_longitude, 
        LA_GUARDIA_LAT, LA_GUARDIA_LON)
    dataframe['pickup_distance_from_j_kennedy'] = np.vectorize(row_modifier)(dataframe.pickup_latitude, dataframe.pickup_longitude, 
        J_KENNEDY_LAT, J_KENNEDY_LON)
    dataframe['dropoff_distance_from_la_guardia'] = np.vectorize(row_modifier)(dataframe.dropoff_latitude, dataframe.dropoff_longitude, 
        LA_GUARDIA_LAT, LA_GUARDIA_LON)
    dataframe['dropoff_distance_from_j_kennedy'] = np.vectorize(row_modifier)(dataframe.dropoff_latitude, dataframe.dropoff_longitude, 
        J_KENNEDY_LAT, J_KENNEDY_LON)
    dataframe['pickup_from_la_guardia'] = dataframe['pickup_distance_from_la_guardia'] < 1.5

    dataframe['pickup_from_la_guardia_zone1'] = dataframe['pickup_distance_from_la_guardia'] < 0.6
    dataframe['pickup_from_la_guardia_zone2'] = \
            (dataframe['pickup_distance_from_la_guardia'] >= 0.6) \
            & (dataframe['pickup_distance_from_la_guardia'] < 1.5)
    dataframe['dropoff_from_la_guardia'] = dataframe['dropoff_distance_from_la_guardia'] < 1.6

    dataframe['dropoff_from_la_guardia_zone1'] = dataframe['dropoff_distance_from_la_guardia'] < 0.75
    dataframe['dropoff_from_la_guardia_zone2'] = \
            (dataframe['dropoff_distance_from_la_guardia'] >= 0.75) \
            & (dataframe['dropoff_distance_from_la_guardia'] < 1.25)
    dataframe['dropoff_from_la_guardia_zone3'] = \
            (dataframe['dropoff_distance_from_la_guardia'] >= 1.25) \
            & (dataframe['dropoff_distance_from_la_guardia'] < 1.6)
    dataframe['pickup_from_j_kennedy'] = dataframe['pickup_distance_from_j_kennedy'] < 1.3

    dataframe['pickup_from_j_kennedy_zone1'] = dataframe['pickup_distance_from_j_kennedy'] < 0.63
    dataframe['pickup_from_j_kennedy_zone2'] = \
            (dataframe['pickup_distance_from_j_kennedy'] >= 0.63) \
            & (dataframe['pickup_distance_from_j_kennedy'] < 1.3)
    dataframe['dropoff_from_j_kennedy'] = dataframe['dropoff_distance_from_j_kennedy'] < 1.5

    dataframe['dropoff_from_j_kennedy_zone1'] = dataframe['dropoff_distance_from_j_kennedy'] < 0.75
    dataframe['dropoff_from_j_kennedy_zone2'] = \
            (dataframe['dropoff_distance_from_j_kennedy'] >= 0.75) \
            & (dataframe['dropoff_distance_from_j_kennedy'] < 1.5)
    return dataframe

In [24]:
dataframe = get_dataframe_for_gradient()

In [97]:
y = dataframe["log_trip_duration"]
X = dataframe.drop(columns=["log_trip_duration"])

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=0)

X_train.reset_index(inplace=True, drop=True)
X_test.reset_index(inplace=True, drop=True)
y_train.reset_index(inplace=True, drop=True)
y_test.reset_index(inplace=True, drop=True)

** Задание 5 (3 балла)**. Обучите и провалидируйте модели на данных из предыдущего пункта, сравните качество между методами по метрикам MSE и $R^2$. Исследуйте влияние параметров `max_iter` и `eta` (`max_iter`, `alpha` и `eta` для Momentum) на процесс оптимизации. Согласуется ли оно с вашими ожиданиями?

In [117]:
categorical_features = ['weekday', 'month', 'dayofyear', 'hour']
numeric_features = [
    'is_anomalous_1', 'is_anomalous_2', 'is_anomalous_3', 'is_anomalous_4', 'empty_roads', 'busy_roads',
    'pickup_from_j_kennedy', 'dropoff_from_j_kennedy', 'pickup_from_la_guardia', 'dropoff_from_la_guardia',
    'log_haversine',
]
column_transformer = ColumnTransformer([
    ('ohe', OneHotEncoder(handle_unknown="ignore"), categorical_features),
    ('scaling', StandardScaler(), numeric_features)
])

pipeline = Pipeline(steps=[
    ('ohe_and_scaling', column_transformer),
    ('regression', LinearRegStochasticGradientDescent(tolerance=1e-20, max_iter=10000, batch_size=10000))
])

In [118]:
model = pipeline.fit(X_train, y_train)

/Users/juliasemavina/anaconda3/lib/python3.7/site-packages/sklearn/preprocessing/data.py:625: DataConversionWarning: Data with input dtype bool, int64, float64 were all converted to float64 by StandardScaler.
  return self.partial_fit(X, y)
/Users/juliasemavina/anaconda3/lib/python3.7/site-packages/sklearn/base.py:465: DataConversionWarning: Data with input dtype bool, int64, float64 were all converted to float64 by StandardScaler.
  return self.fit(X, y, **fit_params).transform(X)


0 42.4492090463017
100 41.840976644916466
200 41.242143829554614
300 40.65262614251741
400 40.07237287050822
500 39.50010891852149


KeyboardInterrupt: 

In [45]:
def rmse(y_true, y_pred):
    error = (y_true - y_pred) ** 2
    return np.sqrt(np.mean(error))
y_pred = model.predict(X_test)
print("Test RMSE = %.4f" % rmse(y_test, y_pred))

/Users/juliasemavina/anaconda3/lib/python3.7/site-packages/sklearn/pipeline.py:605: DataConversionWarning: Data with input dtype bool, int64, float64 were all converted to float64 by StandardScaler.
  res = transformer.transform(X)


Test RMSE = 0.4957


** Задание 6 (2 балла)**. Постройте графики (на одной и той же картинке) зависимости величины функции потерь от номера итерации для полного, стохастического градиентного спусков, а также для полного градиентного спуска с методом Momentum. Сделайте выводы о скорости сходимости различных модификаций градиентного спуска.

Не забывайте о том, что должны получиться *красивые* графики!

In [ ]:
#╰( ͡° ͜ʖ ͡° )つ──☆*:・ﾟ

In [82]:
A = np.array([[1, 2, 3], [4, 5, 6]])
B = np.zeros(3,)

print('A', A.shape)
print('B', B.shape)
# A
np.dot(A, B)
A[[1]]

A (2, 3)
B (3,)


array([[4, 5, 6]])

### Бонус 

** Задание 7 (2 балла)**. Реализуйте линейную регрессию с функцией потерь MSE, обучаемую с помощью метода
[Adam](https://arxiv.org/pdf/1412.6980.pdf) - добавьте при необходимости параметры в класс модели, повторите пункты 5 и 6 и сравните результаты. 

** Задание 8 (2 балла)**. Реализуйте линейную регрессию с функцией потерь
$$ L(\hat{y}, y) = log(cosh(\hat{y} - y)),$$

обучаемую с помощью градиентного спуска.

** Задание 9 (0.01 балла)**.  Вставьте картинку с вашим любимым мемом в этот Jupyter Notebook